In [1]:
%pip install transformers

   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.0 MB 4.2 MB/s eta 0:00:03
   ----- ---------------------------------- 1.3/10.0 MB 4.0 MB/s eta 0:00:03
   ------ --------------------------------- 1.6/10.0 MB 4.0 MB/s eta 0:00:03
   ---------- ----------------------------- 2.6/10.0 MB 3.8 MB/s eta 0:00:02
   -------------- ------------------------- 3.7/10.0 MB 3.8 MB/s eta 0:00:02
   ----------------- ---------------------- 4.5/10.0 MB 3.8 MB/s eta 0:00:02
   ------------------- -------------------- 5.0/10.0 MB 3.6 MB/s eta 0:00:02
   ---------------------- ----------------- 5.8/10.0 MB 3.6 MB/s eta 0:00:02
   -------------------------- ------------- 6.6/10.0 MB 3.7 MB/s eta 0:00:01
   ----------------------------- ---------- 7.3/10.0 MB 3.7 MB/s eta 0:00:01
   --------------------------------- ------ 8.4/10.0 MB 3.8 MB/s eta 0:00:01
   ------------------------------------ --- 9.2/10.0 MB 3.8 MB/s eta 0:00:01
   ---

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
%pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install pdfplumber

   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ----- ---------------------------------- 0.8/5.6 MB 4.8 MB/s eta 0:00:02
   ----------- ---------------------------- 1.6/5.6 MB 4.4 MB/s eta 0:00:01
   ---------------- ----------------------- 2.4/5.6 MB 4.5 MB/s eta 0:00:01
   ---------------------- ----------------- 3.1/5.6 MB 4.2 MB/s eta 0:00:01
   ---------------------------- ----------- 3.9/5.6 MB 4.1 MB/s eta 0:00:01
   --------------------------------- ------ 4.7/5.6 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 5.6/5.6 MB 4.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------- ----------------------------- 0.8/2.9 MB 4.2 MB/s eta 0:00:01
   --------------------- ------------------ 1.6/2.9 MB 4.2 MB/s eta 0:00:01
   ----------------------------- ---------- 2.1/2.9 MB 3.9 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 3.7 MB/s eta 0:00:00
   -----------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
from transformers import pipeline
from transformers import BartTokenizer
from bs4 import BeautifulSoup
import requests
import pdfplumber

def summarize_text(text, max_length=150, min_length=50, do_sample=False):
    """
    Summarizes the input text using a pre-trained Hugging Face transformer model.

    Args:
        text (str): The text to summarize.
        max_length (int): Maximum length of the summary.
        min_length (int): Minimum length of the summary.
        do_sample (bool): Whether or not to sample the output.

    Returns:
        str: The summary of the text.
    """
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=do_sample)
    return summary[0]['summary_text']

from transformers import BartTokenizer

def chunk_text_with_tokenizer(text, tokenizer, max_tokens=450):
    """
    Splits text into smaller chunks using a tokenizer to ensure token limits.

    Args:
        text (str): The input text to be split.
        tokenizer: The Hugging Face tokenizer for the summarization model.
        max_tokens (int): Maximum number of tokens per chunk.

    Returns:
        generator: A generator yielding text chunks.
    """
    tokens = tokenizer(text, return_tensors="pt", truncation=False).input_ids[0]
    chunks = [tokens[i:i + max_tokens] for i in range(0, len(tokens), max_tokens)]
    for chunk in chunks:
        yield tokenizer.decode(chunk, skip_special_tokens=True)

# Example usage with BartTokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")


def summarize_text_in_chunks(text, max_length=150, min_length=50):
    """
    Summarizes large text by splitting it into chunks and summarizing each chunk.

    Args:
        text (str): The input text.
        max_length (int): Maximum length of the summary.
        min_length (int): Minimum length of the summary.

    Returns:
        str: The combined summary of all chunks.
    """
    summaries = []
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

    for chunk in chunk_text_with_tokenizer(text, tokenizer, max_tokens=450):
        try:
            summary = summarize_text(chunk, max_length=max_length, min_length=min_length)
            summaries.append(summary)
        except Exception as e:
            print(f"Error summarizing chunk: {e}")
            print(f"Problematic chunk: {chunk[:200]}...")  # Log the first 200 characters
    return " ".join(summaries)



def summarize_document(file_path, max_length=150, min_length=50):
    """
    Reads a document and summarizes its content.

    Args:
        file_path (str): Path to the document file.
        max_length (int): Maximum length of the summary.
        min_length (int): Minimum length of the summary.

    Returns:
        str: The summary of the document content.
    """
    try:
        with open(file_path, 'r') as file:
            text = file.read()
        return summarize_text_in_chunks(text, max_length=max_length, min_length=min_length)
    except Exception as e:
        return f"Error reading file: {e}"

def summarize_website(url, max_length=150, min_length=50):
    """
    Fetches content from a website and summarizes it.

    Args:
        url (str): The URL of the website to summarize.
        max_length (int): Maximum length of the summary.
        min_length (int): Minimum length of the summary.

    Returns:
        str: The summary of the website content.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extracting text content from the website
        paragraphs = soup.find_all('p')
        content = ' '.join([p.get_text() for p in paragraphs])

        return summarize_text_in_chunks(content, max_length=max_length, min_length=min_length)
    except Exception as e:
        return f"Error fetching website: {e}"

def extract_text_from_pdf(file_path):
    """
    Extracts text from a PDF file using pdfplumber, handling tables and empty pages.

    Args:
        file_path (str): Path to the PDF file.

    Returns:
        str: Extracted text from the PDF.
    """
    try:
        text = ""
        with pdfplumber.open(file_path) as pdf:
            if not pdf.pages:
                return "No pages found in the PDF."

            for i, page in enumerate(pdf.pages):
                try:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
                    elif page.extract_tables():
                        # Extract tables if no text is present
                        tables = page.extract_tables()
                        for table in tables:
                            table_text = "\n".join(["\t".join(row) for row in table if row])
                            text += table_text + "\n"
                        print(f"Extracted table data from page {i + 1}.")
                    else:
                        print(f"Warning: No text or table found on page {i + 1}.")
                except Exception as page_error:
                    print(f"Error processing page {i + 1}: {page_error}")
        
        return text if text.strip() else "No readable text found in the PDF."
    except Exception as e:
        return f"Error extracting text from PDF: {e}"


def summarize_pdf(file_path, max_length=150, min_length=50):
    """
    Extracts text from a PDF file and summarizes its content.

    Args:
        file_path (str): Path to the PDF file.
        max_length (int): Maximum length of the summary.
        min_length (int): Minimum length of the summary.

    Returns:
        str: The summary of the PDF content.
    """
    try:
        # Extract text using pdfplumber
        text = extract_text_from_pdf(file_path)
        if not text.strip() or text == "No text found in the PDF.":
            return "No readable text found in the PDF."
        
        # Summarize the text in chunks
        return summarize_text_in_chunks(text, max_length=max_length, min_length=min_length)
    except Exception as e:
        return f"Error processing PDF file: {e}"


In [20]:
print(summarize_pdf("The Last Light.pdf"))

The Last Light is about a lighthouse in a forgotten village. The villagers had long stopped sailing, and the beacon atop the tower hadn't lit in decades. Yet every day, an old man would climb the narrow spiral stairs to the top. "The light isn’t for the sea, child," he said, "It’s for the ones who are still looking for home"


In [19]:
print(summarize_pdf("Sample.pdf"))

In threaded programming, pthread_join() is used for pausing the main thread until another thread completes/terminates. A process control block (PCB) is a data structure maintained by the operating system for every active process. The program counter plays a vital role in ensuring that processes resume correctly after an interrupt. Calculate the Average waiting time and average turnaround time using the following algorithm:. Shortest Job Remaining Time Next. Highest Response Ratio Next. Average Turnaround Time = 11.5. Average Waiting Time = 5.75. An embedded system is a computer system that is part of a larger device or system. It typically has a dedicated function and operates in real-time environments. Deeply embedded systems are designed to perform very specific, low-level tasks. These systems are typically invisible and fully integrated. Creating processes using the fork() system call in a certain way, will result in a hierarchy of processes. Consider the following hierarchy:Write a

In [21]:
print(summarize_website("https://www.geeksforgeeks.org/learn-data-structures-and-algorithms-dsa-tutorial/?ref=home-articlecards"))

Data Structures and Algorithms (DSA) refer to the study of methods for organizing and storing data. DSA is one of the most important skills that every computer science student must have. Top companies like Google, Microsoft, Amazon, Apple, Meta and many other companies heavily focus on data structures and algorithms during interviews.  Mathematical algorithms are a class of algorithms that solve problems related to mathematical concepts. They are widely used in various fields, including Computer graphics, Numerical analysis, Optimization and Cryptography. Asymptotic notation is a mathematical tool that estimates time based on input size without running the code. Recursion is a programming technique where a function calls itself within its own definition. It is usually used to solve problems that can be broken down into smaller instances of the same problem. Recursion forms the base for many other algorithms such as Tree traversals, Graph traversal, Divide and Conquers Algorithms and Ba